# Definice objektů RLS
Postup:
- vytvoření schématu pro security predicates a security policies
- definice security predicate
- definice security policy
- testování

Předpoklady úspěšného fungování:
- vykonání notebooku RLS_1

## Založení schématu pro RLS objekty
Je dobré, aby objekty RLS měly své vlastní schéma s omezeným přístupem běžných uživatelů.

In [ ]:
use RLS
go

drop security policy if exists rls.policySimpleRls
drop function if exists rls.fnFiltr
drop schema if exists rls
go

create schema rls authorization dbo
go

## Vytvoření security predicate
In-Line Table Valued funkce, která jen definuje podmínky filtrování záznamů. Má právě jeden parametr. Tento parametr nahrazuje sloupec z tabulky, který jsme použili v minulém notebooku.

In [ ]:
use RLS
go

create or alter function rls.fnFiltr(@userName nvarchar(30))
returns table
with SCHEMABINDING
as
return
select 1 as col1 where @userName = USER_NAME() or USER_NAME() = 'BigBoss'
GO

## Vytvoření security policy pro filtrování záznamů</span>

Deklarace spojení tabulky s funkcí. Jedna politika může těchto spojení deklarovat více.

In [ ]:
use RLS
go

create security policy rls.policySimpleRls
add filter predicate rls.fnFiltr(Uzivatel) on dbo.SimpleRls
with (state = on)
go

## Testování výsledku
Zkusíme dotazy v kontextu různých uživatelů

In [ ]:
use RLS
go

select * from dbo.SimpleRls
go

Uživatel **dbo** má prázdný výsledek, ale to jsme čekali. Zkusíme stejný dotaz pro definované uživatele.

In [ ]:
use RLS
go

execute as user = 'Adam'
-- execute as user = 'Bob'
-- execute as user = 'BigBoss'
select * from dbo.SimpleRls
revert
go

Pro zjištění, jak to funguje, si spustíme předchozí kousek kódu v normálním dotazu (copy-paste do query window, zobrazení plánu vykonání).

## Úprava politiky na modifikace</span>
Stávající security politiku předěláme na BLOCK predikát a otestujeme.

In [ ]:
use RLS
go

drop security policy rls.policySimpleRls
GO

create security policy rls.policySimpleRls
add block predicate rls.fnFiltr(Uzivatel) on dbo.SimpleRls after insert
go

In [ ]:
use RLS
go

execute as user = 'Bob'
insert dbo.SimpleRls values ('Eva', 1000000, 'Adam')
select @@error as spatny_insert
insert dbo.SimpleRls values ('Vera', 1000000, 'Bob')
select @@ERROR as dobry_insert
select * from dbo.SimpleRls
revert
go